In [9]:
suppressMessages(library(ArchR))
suppressMessages(library(Seurat))
suppressMessages(library(Signac))
suppressMessages(library(harmony))
suppressMessages(library(dplyr))
suppressMessages(library(cowplot))
suppressMessages(library(harmony))
suppressMessages(library(Nebulosa))
suppressMessages(library(ggpubr))
suppressMessages(library(Ipaper))
suppressMessages(library(phateR))
source("../../utils/stimATAC_analyses_code/R/utils.R")
source("../../utils/stimATAC_analyses_code/R/optMatching_functions.R")

In [10]:
set.seed(42)
getwd()

[1] "/data/scATA/SingleCellOpenChromatin/local/VisiumHeartRevision/IntegrativeAnalysis/Cardiomyocyte/code"

In [11]:
coembed <- readRDS("../data/coembed/coembed.annotation.Rds")
coembed

An object of class Seurat 
230075 features across 59045 samples within 3 assays 
Active assay: RNA (28933 features, 2000 variable features)
 2 other assays present: peaks, GeneActivity
 5 dimensional reductions calculated: pca, umap, harmony, umap_harmony, umap_harmony_v2

In [12]:
coembed <- subset(coembed, annotation %in% c("healthy_CM", "intermediate_CM", "damaged_CM"))

In [13]:
coembed

An object of class Seurat 
230075 features across 55204 samples within 3 assays 
Active assay: RNA (28933 features, 2000 variable features)
 2 other assays present: peaks, GeneActivity
 5 dimensional reductions calculated: pca, umap, harmony, umap_harmony, umap_harmony_v2

In [15]:
## down-sampling, othewise R will fail to create the matrix
coembed.sub <- coembed[, sample(colnames(coembed), size =45000, replace=F)]

In [16]:
obj.atac <- subset(coembed.sub, tech == "ATAC")
obj.rna <- subset(coembed.sub, tech == "RNA")
obj.atac
obj.rna

An object of class Seurat 
230075 features across 8116 samples within 3 assays 
Active assay: RNA (28933 features, 2000 variable features)
 2 other assays present: peaks, GeneActivity
 5 dimensional reductions calculated: pca, umap, harmony, umap_harmony, umap_harmony_v2

An object of class Seurat 
230075 features across 21884 samples within 3 assays 
Active assay: RNA (28933 features, 2000 variable features)
 2 other assays present: peaks, GeneActivity
 5 dimensional reductions calculated: pca, umap, harmony, umap_harmony, umap_harmony_v2

In [17]:
head(coembed.sub@reductions$umap_harmony_v2@cell.embeddings)

,umapharmonyv2_1,umapharmonyv2_2
TCTCTGGTCCGGCTTT-1_2_1_1_1_1_1_1_1_1_1_1_1,-3.5039712,0.678450737
TGCATCCCAAACACGG-1_1_1_1_1_1_1_1_1,0.2775377,-0.677227285
CK380#TGCACCTTCAGGGTTT-1,-1.3187193,-1.160559919
CK173#AACCGATCATTGCCTC-1,-3.6461130,2.045826766
CAATGACTCTCTCGCA-1_1_1_1_1_1_1_1_1_1_1_1,0.5237629,0.310985062
GTTCCGTTCTGGACTA-1_2_1_1_1_1_1_1_1_1_1,0.5970168,-0.005988028


In [18]:
cca_umap_df <- as.data.frame(coembed.sub@reductions$umap_harmony_v2@cell.embeddings)
colnames(cca_umap_df) <- c("UMAP1", "UMAP2")
head(cca_umap_df)

,UMAP1,UMAP2
,<dbl>,<dbl>
TCTCTGGTCCGGCTTT-1_2_1_1_1_1_1_1_1_1_1_1_1,-3.5039712,0.678450737
TGCATCCCAAACACGG-1_1_1_1_1_1_1_1_1,0.2775377,-0.677227285
CK380#TGCACCTTCAGGGTTT-1,-1.3187193,-1.160559919
CK173#AACCGATCATTGCCTC-1,-3.6461130,2.045826766
CAATGACTCTCTCGCA-1_1_1_1_1_1_1_1_1_1_1_1,0.5237629,0.310985062
GTTCCGTTCTGGACTA-1_2_1_1_1_1_1_1_1_1_1,0.5970168,-0.005988028


In [19]:
options(repr.plot.height = 6, repr.plot.width = 6)

df_cell_pairing <- cell_pairing(ATACpcs = obj.atac@reductions$harmony@cell.embeddings,
                               RNApcs = obj.rna@reductions$harmony@cell.embeddings,
                               cca_umap_df = cca_umap_df,
                               nCores = 80)


Attaching package: ‘igraph’


The following object is masked from ‘package:FNN’:

    knn


The following objects are masked from ‘package:dplyr’:

    as_data_frame, groups, union


The following object is masked from ‘package:GenomicRanges’:

    union


The following object is masked from ‘package:IRanges’:

    union


The following object is masked from ‘package:S4Vectors’:

    union


The following objects are masked from ‘package:BiocGenerics’:

    normalize, path, union


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union



Attaching package: ‘pracma’


The following objects are masked from ‘package:Ipaper’:

    fliplr, flipud, fprintf


The following objects are masked from ‘package:magrittr’:

    and, mod, or


The following objects are masked from ‘package:Matrix’:

    expm, lu, tril, triu




In [ ]:
sel_cells <- c(df_cell_pairing$ATAC, df_cell_pairing$RNA)
coembed.sub <- coembed[, sel_cells]

options(repr.plot.height = 5, repr.plot.width = 10)
DimPlot(coembed.sub, reduction = "umap_harmony_v2", group.by = "annotation", split.by = "tech")

In [ ]:
df_cell_pairing$cell_name <- paste0("cell-", 1:nrow(df_cell_pairing))

In [ ]:
saveRDS(df_cell_pairing, "../data/coembed/ATAC_RNA_matching.rds")

In [ ]:
sessionInfo()